In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import h5py

def inspect_h5_file(file_path):
    with h5py.File(file_path, 'r') as file:
        print("Keys in the HDF5 file:")
        print(list(file.keys()))

inspect_h5_file('/content/drive/MyDrive/Deep Learning/Lab2/Te.h5')
inspect_h5_file('/content/drive/MyDrive/Deep Learning/Lab2/Tr.h5')

Keys in the HDF5 file:
['images', 'labels']
Keys in the HDF5 file:
['images', 'labels']


1. Load and preprocess the data.

In [3]:
import numpy as np
def load_data(train_path, test_path):
    with h5py.File(train_path, 'r') as train_file:
        X_train = np.array(train_file['images'][:])
        y_train = np.array(train_file['labels'][:])
    with h5py.File(test_path, 'r') as test_file:
        X_test = np.array(test_file['images'][:])
        y_test = np.array(test_file['labels'][:])
    return X_train, y_train, X_test, y_test

In [4]:
X_train, y_train, X_test, y_test = load_data('/content/drive/MyDrive/Deep Learning/Lab2/Tr.h5', '/content/drive/MyDrive/Deep Learning/Lab2/Te.h5')

In [5]:
def preprocess_data(X):
    X_flatten = X.reshape(X.shape[0], -1).T
    X_norm = X_flatten / 255.
    return X_norm

X_train = preprocess_data(X_train)
X_test = preprocess_data(X_test)

In [6]:
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)

Shape of X_train: (49152, 250)
Shape of y_train: (250,)


In [7]:
def one_hot_encoding(y, num_classes):
    m = y.shape[0]
    one_hot = np.zeros((num_classes, m))
    one_hot[y, np.arange(m)] = 1
    return one_hot

In [8]:
# Assuming the labels range from 0 to num_classes-1
num_classes = np.max(y_train) + 1
y_train_one_hot = one_hot_encoding(y_train, num_classes)  # Shape should be (5, 2626)
y_test_one_hot = one_hot_encoding(y_test, num_classes)    # Shape should be (5, number_of_test_samples)

print("Shape of y_train_one_hot:", y_train_one_hot.shape)

Shape of y_train_one_hot: (5, 250)


2. Implement the softmax activation function.

In [9]:
def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=0, keepdims=True))
    return expZ / np.sum(expZ, axis=0, keepdims=True)

3. Define the cost function for multi-class classification.

In [10]:
def compute_cost(AL, Y):
    m = Y.shape[1]
    cost = -np.sum(Y * np.log(AL + 1e-8)) / m
    return cost

4. Implement the gradient descent algorithm to update the model parameters.

In [11]:
def gradient_descent(X, Y, learning_rate, num_iterations):
    n_x = X.shape[0]
    n_y = Y.shape[0]
    m = X.shape[1]

    W = np.random.randn(n_y, n_x) * 0.01
    b = np.zeros((n_y, 1))

    for i in range(num_iterations):
        Z = np.dot(W, X) + b
        A = softmax(Z)

        cost = compute_cost(A, Y)

        dZ = A - Y
        dW = np.dot(dZ, X.T) / m
        db = np.sum(dZ, axis=1, keepdims=True) / m

        W -= learning_rate * dW
        b -= learning_rate * db

        if i % 100 == 0:
            print(f"Cost after iteration {i}: {cost}")

    return W, b

5. Test the model on the test set and evaluate its performance.

In [12]:
def predict(X, W, b):
    Z = np.dot(W, X) + b
    AL = softmax(Z)
    return np.argmax(AL, axis=0)

In [13]:
# Train the model on lr = 0.01 and iterations = 1000
W, b = gradient_descent(X_train, y_train_one_hot, learning_rate=0.01, num_iterations=1000)

# Test the model
predictions = predict(X_test, W, b)
accuracy = np.mean(predictions == np.argmax(y_test_one_hot, axis=0))
print(f"Accuracy on test set: {accuracy * 100}%")

Cost after iteration 0: 1.9000027848487622
Cost after iteration 100: 2.4734939063873496
Cost after iteration 200: 0.04599522676439637
Cost after iteration 300: 0.02069646663811797
Cost after iteration 400: 0.014854001473955096
Cost after iteration 500: 0.012105233038390707
Cost after iteration 600: 0.01042707132863783
Cost after iteration 700: 0.009258221954256098
Cost after iteration 800: 0.00837923676614967
Cost after iteration 900: 0.007684873334541076
Accuracy on test set: 34.0%


In [14]:
# Train the model on lr = 0.001 and iterations = 2000
W, b = gradient_descent(X_train, y_train_one_hot, learning_rate=0.001, num_iterations=2000)

# Test the model
predictions = predict(X_test, W, b)
accuracy = np.mean(predictions == np.argmax(y_test_one_hot, axis=0))
print(f"Accuracy on test set: {accuracy * 100}%")

Cost after iteration 0: 2.3211914280536674
Cost after iteration 100: 0.843781343457965
Cost after iteration 200: 0.6031217155519147
Cost after iteration 300: 0.4674651821767941
Cost after iteration 400: 0.3792550190607334
Cost after iteration 500: 0.3174214291584311
Cost after iteration 600: 0.27180191219583566
Cost after iteration 700: 0.23685189052946418
Cost after iteration 800: 0.20929354923355356
Cost after iteration 900: 0.18706594548942673
Cost after iteration 1000: 0.16880671959633844
Cost after iteration 1100: 0.15357726716682374
Cost after iteration 1200: 0.14070874638334543
Cost after iteration 1300: 0.1297115049833948
Cost after iteration 1400: 0.1202192313052132
Cost after iteration 1500: 0.11195295313241337
Cost after iteration 1600: 0.10469694127196463
Cost after iteration 1700: 0.09828209976014687
Cost after iteration 1800: 0.09257425457270411
Cost after iteration 1900: 0.08746574542636544
Accuracy on test set: 40.0%
